#### pandas data 호출

In [2]:
import numpy as np
import pandas as pd

#### Discretization and binning(개별화와 양자화)
연속된 data는 종종:
- 개별로 분할하거나
- 분석을 위한 그룹(bin)으로 나눈다.

pd.cut()
- bin으로 나눌 구간은 ( , ]으로 정의되므로 첫 항은 빠진다. 따라서 일반적으로 +- 5% 정도 범위를 늘려서 계산한다.

In [3]:
path = 'C:/Users/tmznq/workspace/ml_scratch/dataset/titanic/train.csv'
path2 = 'C:/Users/tmznq/workspace/ml_scratch/dataset/titanic/test.csv'
path3 = 'C:/Users/tmznq/workspace/ml_scratch/dataset/titanic/gender_submission.csv'
df = pd.read_csv(path)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)

df3.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


### indexing 참고할 것.
df[0] : 점색인. 열
df[:7] : 연속색인. 행
df[['col1', 'col2']] : 열색인, boolean 색인

#### merge, join, concat 복습

#### 1. removing duplicates
- 중복된 row를 식별하고 이를 제거하는 방법을 학습한다.
- df.duplicated() : df.drop_duplicates()의 하위 개념으로 함수 적용 시, 중복인지 아닌지 여부를 boolean Series로 반환.
- subset : column label 혹은 sequence of labels. 중복을 식별할 열을 지정.
- keep
- inplace

In [ ]:
df.drop_duplicates = df[~df.duplicated()] : 같은 의미. why? boolean Series에 의한 indexing

In [ ]:
df.drop_duplicates(['k1']) # 1개의 열을 고르더라도 subset으로 다루기 때문에 여러 열을 사용하듯이 지정해야함.

#### 2. transforming, mapping

#### mapping
- 'A' 열의 설명을 위한 새로운 column을 추가하려고 할 때.
- 'A'의 각 열의 key을 값으로하는 열을 추가하고 싶을 때. map 함수 사용
- map : vector(Series)의 각 성분별 함수 결과 또는 각 성분을 key로하는 value를 리턴한다. df에서는 사용불가

ex) data['col1'] = data['A'].map(str, lower).map(dict) # dict의 key는 'A'의 각 열과 동일하다.

#### replace value
- 결측치 대체
- 

In [1]:
data = Series([1, -999, 3, -999, 5])
data

NameError: name 'Series' is not defined

In [ ]:
data.replace(-999, np.nan)
data.replace([-999, 5], [np.nan, 0])

#### renaming axis indexes

#### Discretization and binning(개별화와 양자화)
연속된 data는 종종:
- 개별로 분할하거나
- 분석을 위한 그룹(bin)으로 나눈다.

pd.cut()
- bin으로 나눌 구간은 ( , ]으로 정의되므로 첫 항은 빠진다. 따라서 일반적으로 +- 5% 정도 범위를 늘려서 계산한다.

In [1]:
# 치환과 임의 샘플링
# 임의의 데이터에서 train, test data는 random sampling해야 한다.

In [ ]:
# 더미변수 
# str data를 숫자로 표현했을 때 발생하는 문제점 보완.

## 문자열 다루기

In [ ]:
# 정규표현식
# 자주 까먹음

#### 1. removing duplicates
- 중복된 row를 식별하고 이를 제거하는 방법을 학습한다.
- df.duplicated() : df.drop_duplicates()의 하위 개념으로 함수 적용 시, 중복인지 아닌지 여부를 boolean Series로 반환.
- subset : column label 혹은 sequence of labels. 중복을 식별할 열을 지정.
- keep
- inplace

In [ ]:
df.drop_duplicates = df[~df.duplicated()] : 같은 의미. why? boolean Series에 의한 indexing

In [ ]:
df.drop_duplicates(['k1']) # 1개의 열을 고르더라도 subset으로 다루기 때문에 여러 열을 사용하듯이 지정해야함.

#### 2. transforming, mapping

#### mapping
- 'A' 열의 설명을 위한 새로운 column을 추가하려고 할 때.
- 'A'의 각 열의 key을 값으로하는 열을 추가하고 싶을 때. map 함수 사용
- map : vector(Series)의 각 성분별 함수 결과 또는 각 성분을 key로하는 value를 리턴한다. df에서는 사용불가

ex) data['col1'] = data['A'].map(str, lower).map(dict) # dict의 key는 'A'의 각 열과 동일하다.

#### 3. replace value
- 결측치 대체
- 

In [1]:
data = Series([1, -999, 3, -999, 5])
data

NameError: name 'Series' is not defined

In [ ]:
data.replace(-999, np.nan)
data.replace([-999, 5], [np.nan, 0])

#### 4. renaming axis indexes

#### 5. Discretization and binning(개별화와 양자화)
연속된 data는 종종:
- 개별로 분할하거나
- 분석을 위한 그룹(bin)으로 나눈다.

pd.cut()
- bin으로 나눌 구간은 ( , ]으로 정의되므로 첫 항은 빠진다. 따라서 일반적으로 +- 5% 정도 범위를 늘려서 계산한다.

#### 6. filtering outliers
- df.describe()

In [ ]:
# 통계량 boolean indexing 적용해보기
ex) data[['col1', 'col2']][:5][True, True, False, False, False] 

In [ ]:
# Series 적용해보기
col = data[3]
col[np.abs(col) > 3] 

- df.any

In [4]:
arr = np.array([0, -5, 2, 7])
arr > 0

array([False, False,  True,  True])

In [ ]:
l = [0, -5, 2, 7]

b = list(map(lambda x: x>0, l)) # list boolean은 이런식으로...?
any(b) # 1개라도 True면 True

In [12]:
all(b) # 전부 True가 아니라면 False

False

In [ ]:
(np.abs(data) > 3).any(axis = 0) # np.abs(data) > 3는 df으로 boolean indexing 되서 반환된다.

In [ ]:
data[(np.abs(data) > 3).any(axis = 0)] # boolean indexing으로 True인 데이터만 df으로 반환.

In [ ]:
data[np.abs(data) > 3] = np.sign(data) * 3 # 왼쪽의 data는 data size가 유지됨.
data.describe()

In [1]:
# 치환과 임의 샘플링
# 임의의 데이터에서 train, test data는 random sampling해야 한다.

In [ ]:
# 더미변수 
# str data를 숫자로 표현했을 때 발생하는 문제점 보완.

## 문자열 다루기

In [ ]:
# 정규표현식
# 자주 까먹음